## 1. Import Modules and Data
BERT can be fine-tined on Stanford Sentiment Treebank-2(SST2) dataset for text classification task. More info about SST2 can be found [here](https://huggingface.co/datasets/stanfordnlp/sst2).

In [ ]:
import torch
from transformers import BertTokenizer, AdamW
from data import load_data
from modules.bert import BERTTextClassifier, ScheduledOptim
import config 
# load sst-2
tokenizer, train_dataloader, valid_dataloader = load_data(
    name="sst2",
    loading_ratio=1,  # load 100% sst-2 data
    num_proc=4,  # use 4 processes
    splits=["train", "validation"]  # load train and validation dataset
)

# check one batch shape
for batch in train_dataloader:
    input_ids, attention_mask, labels = batch
    print(input_ids.shape, attention_mask.shape, labels.shape)
    break


## 2. Build Model and Load from Pre-trained
Build a BERT text classification model which inherits from the BERT class and add a binary linear classification layer at the end of the structure.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# load pretrained model
model = BERTTextClassifier.from_pretrained(
    config.pretrained_path,
    num_frozen_layers=0,  # frozen layer. Here do not froze
)
model = model.to(device)

print(model)


Number of trainable parameters: 108.89M
BERTTextClassifier(
  (embedding): BERTEmbedding(
    (token): TokenEmbedding(30522, 768, padding_idx=0)
    (position): PositionalEmbedding(512, 768)
    (segment): SegmentEmbedding(2, 768, padding_idx=0)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): ModuleList(
    (0): TransformerBlock(
      (self_attn): MultiheadAttention(
        (attention): ScaledDotProductAttention()
        (w_q): Linear(in_features=768, out_features=768, bias=True)
        (w_k): Linear(in_features=768, out_features=768, bias=True)
        (w_v): Linear(in_features=768, out_features=768, bias=True)
        (w_concat): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_1): LayerNorm()
      (ffn): PositionwiseFeedForward(
        (linear1): Linear(in_features=768, out_features=3072, bias=True)
        (linear2): Linear(in_features=3072, out_features=768, bias=True)
        (d

## 3. Train Model

### 3.1 Optimizer and Scheduler
ScheduledOptim class is a wrapper for an optimizer that implements a learning rate scheduling strategy inspired by the Transformer paper (Attention Is All You Need). It adjusts the learning rate using a warm-up and decay mechanism to stabilize training. You can check modules/optim_schedule.py for deeper understanding.

Original paper shows that they use Adam with learning rate of 1e-4, β1 = 0.9, β2 = 0.999, L2 weight decay of 0.01, learning rate warmup over the first 10,000 steps, and linear decay of the learning rate in the pretraining.

For fine-tuning, most model hyperparameters are the same as in pre-training, with the exception of the batch size, learning rate, and number of training epochs.
The dropout probability was always kept at 0.1. The optimal hyperparameter values are task-specific.

The learing rate schedule is shown as below, and initial lr is set as 2e-5.

### 3.2 Train Loop

In [3]:
import torch
from transformers import BertTokenizer
from torch.optim import AdamW

from data import load_data
from modules.bert import BERTTextClassifier, ScheduledOptim
import config  

# finetune
pad_idx = 0
def split_batch(batch):  
    tokens, attention_mask, labels = batch 
    attention_mask = (tokens != pad_idx).long()  
    input_ids = tokens.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)
    return input_ids, attention_mask, labels

from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
@torch.no_grad()
def evaluate(model, clf_criterion, dataloader):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    for batch in tqdm(dataloader, desc="Evaluating"):
        input_ids, attention_mask, gt = split_batch(batch)

        clf_logits = model(input_ids)  # get classfier logits
        
        # print(f"evaluate_clf_logits: {clf_logits}")

        loss = clf_criterion(clf_logits, gt)
        total_loss += loss.item()

        preds = torch.argmax(clf_logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(gt.cpu().numpy())
        # get predictions and GT
        # print(f"Predictions: {preds.cpu().numpy()}")
        # print(f"Ground Truth: {gt.cpu().numpy()}")

    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)

    return avg_loss, accuracy


In [4]:
def train(epoch, model, scheduled_optimizer, dataloader):
    model.train()
    total_loss = 0
    scheduled_optimizer.zero_grad()  

    for batch in tqdm(dataloader, desc=f"Training Epoch {epoch}"):
        input_ids, attention_mask, labels = split_batch(batch)
        
        # single sentence, requiring no segment info
        segment_info = torch.zeros(input_ids.size(0), input_ids.size(1))

        input_ids = input_ids.to(device).long()  
        attention_mask = attention_mask.to(device).long()  
        segment_info = segment_info.to(device).long()  
        
        
        loss, logits = model(input_ids, segment_info=segment_info, labels=labels)
        
        loss.backward()
        # grad-clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # update step and lr
        scheduled_optimizer.step_and_update_lr() 

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [5]:
def training_loop(model, train_dataloader, valid_dataloader, optimizer, criterion, num_epochs, warmup_steps):

    scheduled_optimizer = ScheduledOptim(
        optimizer=optimizer,
        n_warmup_steps=warmup_steps,
        total_steps=len(train_dataloader) * num_epochs
    )

    for epoch in range(num_epochs):
        # train
        avg_train_loss = train(
            epoch + 1, model, scheduled_optimizer, train_dataloader
        )

        # valid
        avg_valid_loss, avg_acc = evaluate(model, criterion, valid_dataloader)

        print(
            f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f},",
            f"Validation Loss: {avg_valid_loss:.4f}, Accuracy: {avg_acc * 100:.2f}",
        )

        # save checkpoint
        checkpoint_path = config.checkpoint_dir / f"bert_clf_{epoch + 1}.pth"
        torch.save(
            {
                "epoch": epoch + 1,
                "model": model.state_dict(),
                "optimizer": scheduled_optimizer._optimizer.state_dict(),
                "scheduler": scheduled_optimizer.n_current_steps,
            },
            checkpoint_path,
        )


# AdamW optimizer
optimizer = AdamW(model.parameters(), lr=config.FinetuningConfig.lr, weight_decay=config.FinetuningConfig.weight_decay)

training_loop(model, train_dataloader, valid_dataloader, optimizer, torch.nn.CrossEntropyLoss(), config.FinetuningConfig.n_epoch, config.FinetuningConfig.warmup_steps)

Training Epoch 1:   0%|          | 0/2105 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 1/3, Training Loss: 0.4088, Validation Loss: 0.2562, Accuracy: 89.56


Training Epoch 2:   0%|          | 0/2105 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 2/3, Training Loss: 0.2057, Validation Loss: 0.2360, Accuracy: 91.51


Training Epoch 3:   0%|          | 0/2105 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 3/3, Training Loss: 0.1456, Validation Loss: 0.2189, Accuracy: 91.97
